In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

import time
rcParams['font.family'] = 'serif'
% matplotlib

In [ ]:
data = np.loadtxt('DL07_milimill_HOD.csv')
IDs = data[:,0]
fofid = data[:,1]
stellarmass = data[:,2]    # 10^10 /h Msun
centralmvir = data[:,3]    # 10^10 /h Msun
Type = data[:,4]

In [ ]:
def ToClass(X, NBIN, XMIN,XMAX):
    labels = (np.array(X-XMIN)/(XMAX-XMIN)*NBIN).astype(int)
    return labels

In [ ]:
max_masslog = np.max(np.log10(centralmvir) + 10) + 1e-10
min_masslog = np.min(np.log10(centralmvir) + 10)

NBIN = 30
BinSize = (max_masslog - min_masslog)/NBIN

log_centralmvir = np.log10(centralmvir) + 10
labels = ToClass(log_centralmvir, NBIN, min_masslog, max_masslog)
bins = np.linspace(min_masslog, max_masslog, NBIN) + BinSize *.5

In [ ]:
min_masslog, max_masslog

In [ ]:
start_time = time.time()
HOD = np.zeros(NBIN)
Centrals = np.zeros(NBIN)
Sats = np.zeros(NBIN)

for i in range(NBIN):
    
    idx = np.where(labels == i)[0]
    Halos = np.where(Type[idx] == 0)[0] # = Centrals
    
    mask_HOD = stellarmass[idx] > .1
    mask_Cent = (stellarmass[idx] > .1) & (Type[idx] == 0)
    mask_Sats = (stellarmass[idx] > .1) & (Type[idx] != 0)
    
    HOD[i] = (len((stellarmass[idx])[mask_HOD]) +1e-10) / float(len(Halos))
    Centrals[i] = (len((stellarmass[idx])[mask_Cent]) +1e-10) / float(len(Halos))
    Sats[i] = (len((stellarmass[idx])[mask_Sats]) +1e-10) / float(len(Halos))
    
    print len(Halos), HOD[i]*len(Halos)
    
print("--- %3fs: loaded ---" % (time.time() - start_time))  

In [ ]:
np.savetxt("HOD.txt", np.array([bins, HOD, Centrals, Sats]).T)

In [ ]:
#plt.figure(figsize = [6,6])
plt.plot(bins, np.log10(HOD), 'k.', label = 'All')
plt.plot(bins, np.log10(Centrals), 'r--', label = 'Centrals')
plt.plot(bins, np.log10(Sats), 'b--', label = 'Sats')
#plt.annotate(r"$M > 10^9 M_{\odot}$", xy=(80, 270), xycoords='figure pixels')
plt.xlabel(r"$log(M_h / h^{-1}M_{\odot})$")
plt.ylabel(r"$log(N)$")
plt.ylim(-2,2)
plt.xlim(10,15)
plt.minorticks_on()
plt.legend(loc='upper left', frameon = False, prop={'size':10})
plt.show()